# Constrained Linear Regression Model #

### Sum of Betas equal to one ####

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visuals
import seaborn as sns
import matplotlib.pyplot as plt

# Statistics
import statsmodels.api as sm 
from scipy.stats import t
from scipy.stats import f
from scipy.stats import chi2

# Pretty Notation
from IPython.display import display, Math

In [2]:
# The Constrained OLS implies a monothonic transformation

display(Math(r"\beta=(X^⊤X)^{-1}(X^⊤Y)-P"))
display(Math(r"P=\frac{\iota^⊤(X^⊤X)^{-1}(X^⊤Y)-1}{\iota^⊤(X^⊤X)^{-1}\iota}(X^⊤X)^{-1}\iota"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
# Import Data

df_data = pd.read_csv(rf"..\additional_data\advertising.csv")

df_data = df_data.drop('Unnamed: 0', axis=1)

# Use Logs
df_data.replace(0, np.nan, inplace=True)
df_data.dropna(inplace=True)
df_data = np.log(df_data)

df_data

,TV,radio,newspaper,sales
0,5.438514,3.632309,4.237001,3.095578
1,3.795489,3.671225,3.808882,2.341806
2,2.844909,3.826465,4.238445,2.230014
3,5.020586,3.720862,4.069027,2.917771
4,5.197391,2.379546,4.067316,2.557227
...,...,...,...,...
195,3.642836,1.308333,2.624669,2.028148
196,4.545420,1.589235,2.091864,2.272126
197,5.176150,2.230014,1.856298,2.549445
198,5.647565,3.737670,4.192680,3.238678


In [191]:
# Let us check the correlations

correlation = df_data.corr()

correlation

,TV,radio,newspaper,sales
TV,1.000000,0.022887,0.051412,0.862218
radio,0.022887,1.000000,0.173105,0.452618
newspaper,0.051412,0.173105,1.000000,0.155127
sales,0.862218,0.452618,0.155127,1.000000


In [51]:
# The matrix

df_data['intercept'] = 1

Y_Vector = df_data["sales"]
Information_Matrix = df_data[['intercept', 'TV', 'radio', 'newspaper']]

In [91]:
#Model specification
model = sm.OLS(
    Y_Vector, 
    sm.add_constant(Information_Matrix)
    )   
     
#the results of the model
results = model.fit() 
    
#The Parameters
R2 = results.rsquared  

#here we check the summary
print(results.summary())   

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     894.7
Date:                Fri, 14 Mar 2025   Prob (F-statistic):          1.08e-113
Time:                        20:23:25   Log-Likelihood:                 161.28
No. Observations:                 199   AIC:                            -314.6
Df Residuals:                     195   BIC:                            -301.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.4006      0.046      8.683      0.0

In [61]:
# Let us calculate the betas and the penalization
Information_Matrix_T = Information_Matrix.transpose()

# Information Matrix Squared
A = Information_Matrix_T.dot(Information_Matrix)

# X*Y
b = Information_Matrix_T.dot(Y_Vector)

In [67]:
# Calculate the penalization components

n = len(Y_Vector)
k = 4

iota = np.ones(k).transpose()
iota_T = iota.transpose()
A_inv = np.linalg.inv(A)

In [69]:
# The Penalization

P = (((iota_T @ A_inv @ b) - 1)/(iota_T @ A_inv @ iota)) * (A_inv @ iota)

print(P)

[-0.08209611  0.01083226  0.00412618  0.00546697]


In [71]:
# Now Obtain the betas 

betas = (A_inv @ b)

print(betas)

[0.40064457 0.34981299 0.17188449 0.01598725]


In [73]:
# Now the adjusted betas

betas_adj = betas - P

print(betas_adj)

[0.48274068 0.33898073 0.16775831 0.01052028]


In [77]:
# Obtain the fitted values

y_fitted = Information_Matrix @ betas_adj

df_data['y_fitted'] = y_fitted

df_data

,TV,radio,newspaper,sales,intercept,expected_returns,y_fitted
0,5.438514,3.632309,4.237001,3.095578,1,2.980217,2.980217
1,3.795489,3.671225,3.808882,2.341806,1,2.425287,2.425287
2,2.844909,3.826465,4.238445,2.230014,1,2.133621,2.133621
3,5.020586,3.720862,4.069027,2.917771,1,2.851635,2.851635
4,5.197391,2.379546,4.067316,2.557227,1,2.686534,2.686534
...,...,...,...,...,...,...,...
195,3.642836,1.308333,2.624669,2.028148,1,1.964688,1.964688
196,4.545420,1.589235,2.091864,2.272126,1,2.312165,2.312165
197,5.176150,2.230014,1.856298,2.549445,1,2.630988,2.630988
198,5.647565,3.737670,4.192680,3.238678,1,3.068290,3.068290


In [83]:
# Obtain the errors

df_data['residuals'] = df_data['sales'] - df_data['y_fitted']

df_data

,TV,radio,newspaper,sales,intercept,expected_returns,y_fitted,residuals
0,5.438514,3.632309,4.237001,3.095578,1,2.980217,2.980217,0.115361
1,3.795489,3.671225,3.808882,2.341806,1,2.425287,2.425287,-0.083481
2,2.844909,3.826465,4.238445,2.230014,1,2.133621,2.133621,0.096393
3,5.020586,3.720862,4.069027,2.917771,1,2.851635,2.851635,0.066135
4,5.197391,2.379546,4.067316,2.557227,1,2.686534,2.686534,-0.129307
...,...,...,...,...,...,...,...,...
195,3.642836,1.308333,2.624669,2.028148,1,1.964688,1.964688,0.063461
196,4.545420,1.589235,2.091864,2.272126,1,2.312165,2.312165,-0.040039
197,5.176150,2.230014,1.856298,2.549445,1,2.630988,2.630988,-0.081543
198,5.647565,3.737670,4.192680,3.238678,1,3.068290,3.068290,0.170389


In [93]:
# Calculate the R-Squared

# RSS
SSR_restr = np.sum((Y_Vector - y_fitted)**2)

#TSS
SST = np.sum((Y_Vector - np.mean(Y_Vector))**2)

# R_Squared
R2_restr = 1 - (SSR_restr / SST)

print(f"The R2 of the unconstrained regression: {R2}")
print(f"The R2 of the constrained regression: {R2_restr}")

The R2 of the unconstrained regression: 0.9322706060910606
The R2 of the constrained regression: 0.9311590717243388


In [117]:
# Calculate Significance of the Constrained OLS

Residuals_Variance = SSR_restr/(n - k)

Covariance_Matrix = (Residuals_Variance)*A_inv

Beta_Standards_Errors = np.sqrt(Covariance_Matrix.diagonal())

T_Values = betas_adj/Beta_Standards_Errors

Beta_Lower_Limit = betas_adj - 1.96*Beta_Standards_Errors
Beta_Upper_Limit = betas_adj + 1.96*Beta_Standards_Errors

Proof_DF = pd.DataFrame(
    {
     "Betas": betas_adj,
     "Std": Beta_Standards_Errors,
     "T_Values": T_Values, 
     "Beta_Inferior_Limit": Beta_Lower_Limit, 
     "Beta_Superior_Limit": Beta_Upper_Limit
     }
    )

Proof_DF["p-values"] = 2*(t.sf(
    abs(Proof_DF.T_Values), 
    n-k,
    ).round(3)
    )

Proof_DF

,Betas,Std,T_Values,Beta_Inferior_Limit,Beta_Superior_Limit,p-values
0,0.482741,0.046518,10.377520,0.391566,0.573916,0.000
1,0.338981,0.007739,43.802643,0.323813,0.354149,0.000
2,0.167758,0.007686,21.825578,0.152693,0.182823,0.000
3,0.010520,0.008133,1.293525,-0.005420,0.026461,0.198


In [183]:
display(Math(r"F=\frac{\left(SSR_{const}-SSR_{OLS}\right)/m}{SSR_{ols}/n-k}"))

<IPython.core.display.Math object>

In [133]:
# Test of validity of the constraints

# Obtain the OLS RSS
residuals_ols = results.resid
SSR_ols = np.sum(residuals_ols ** 2)

# Calculate the F-Stat

# Number of restrictions
m = 1  

# F-statistic
F_stat = ((SSR_restr - SSR_ols) / m) / (SSR_ols / (n - k))

F_stat

3.2002235514193953

In [139]:
# The p-value

p_value = 1 - f.cdf(F_stat, m, n - k)

print("p-value:", p_value)

# Conclusion
if p_value < 0.05:
    print("Reject the null hypothesis: The constraint is NOT valid.")
else:
    print("Fail to reject the null: The constraint is valid.")

p-value: 0.07518040594853459
Fail to reject the null: The constraint is valid.


In [163]:
# Let us make the Wald Test

# Define the Restrictions R and q

R = np.array([[1, 1, 1, 1]])  
q = np.array([[1]])

# get the variances of the OLS betas
var_beta_hat = results.cov_params()

var_beta_hat

,intercept,TV,radio,newspaper
intercept,0.002129,-2.638517e-04,-1.262170e-04,-0.000157
TV,-0.000264,5.892232e-05,-8.322138e-07,-0.000003
radio,-0.000126,-8.322138e-07,5.812555e-05,-0.000011
newspaper,-0.000157,-2.984131e-06,-1.059103e-05,0.000065


In [175]:
# Compute Wald statistic
diff = R@betas - q  # (m x 1)
middle = R @ var_beta_hat @ R.T  # (m x m), scalar here since m=1

# Wald statistic (scalar)
W = diff.T @ np.linalg.inv(middle) @ diff
print("Wald statistic:", W)

Wald statistic: [[3.20022355]]


In [177]:
# p-value from Chi-squared distribution with m degrees of freedom
m = R.shape[0]  # Number of restrictions
p_value = 1 - chi2.cdf(W, df=m)

print("p-value:", p_value)

# Conclusion
if p_value < 0.05:
    print("Reject the null hypothesis: The constraint is NOT valid.")
else:
    print("Fail to reject the null hypothesis: The constraint is valid.")

p-value: [[0.07362821]]
Fail to reject the null hypothesis: The constraint is valid.
